In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [4]:
class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(BayesianLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        # Define parameters for weight mean and standard deviation
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features))
        self.weight_rho = nn.Parameter(torch.Tensor(out_features, in_features))

        # Initialize parameters
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.normal_(self.weight_mu, mean=0, std=0.1)
        nn.init.constant_(self.weight_rho, -5)

    def forward(self, x):
        # During training, sample random weights from the distribution
        if self.training:
            epsilon = torch.randn_like(self.weight_mu)
            weight = self.weight_mu + torch.log(1 + torch.exp(self.weight_rho)) * epsilon
        else:
            # During evaluation, use the mean of the distribution
            weight = self.weight_mu

        return F.linear(x, weight)

class BayesianNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BayesianNetwork, self).__init__()
        self.fc1 = BayesianLinear(input_size, hidden_size)
        self.fc2 = BayesianLinear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [5]:
# Example usage
input_size = 10
hidden_size = 5
output_size = 1

bayesian_net = BayesianNetwork(input_size, hidden_size, output_size)
input_data = torch.randn(1, input_size)  # Replace this with your actual input data

output = bayesian_net(input_data)
print("Model Output:", output)

Model Output: tensor([[0.0115]], grad_fn=<MmBackward0>)
